# Scrape Data From JustWatch

In [2]:
# !pip install JustWatch

  Created wheel for JustWatch: filename=JustWatch-0.5.1-py3-none-any.whl size=3745 sha256=34136a9ee80f9dd4a1d4856f334a44c71baae7389c728e9ebd8ecba9c1e132a9
  Stored in directory: c:\users\ghall\appdata\local\pip\cache\wheels\76\51\c1\482d17ebd8878cab098015348f6dad6c951a4e68ecd046133c
Successfully built JustWatch
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

selenium 4.4.0 requires urllib3[secure,socks]~=1.26, but you'll have urllib3 1.25.11 which is incompatible.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you'll have markupsafe 2.0.1 which is incompatible.
google-api-core 1.22.2 requires protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.


In [230]:
# import libraries
import pandas as pd
import time

# import JustWatchAPI
from justwatch import JustWatch

In [160]:
# instantiate JustWatch class
just_watch = JustWatch(country='US')

## Helper Function

In [300]:
def create_show_dict(show_title, show_id):    
    # show dictionary
    show_dict = {}

    # keys and values
    show_dict['jw_entity_id'] = show_title['jw_entity_id']
    show_dict['id'] = show_title['id']
    show_dict['title'] = show_title['title']
    
    if 'poster' in show_title.keys():
        show_dict['poster'] = show_title['poster']
    else:
        show_dict['poster'] = None
    
    if 'short_description' in show_title.keys():
        show_dict['description'] = show_title['short_description']
    else:
        show_dict['description'] = None
        
    if 'original_release_year' in show_title.keys():
        show_dict['release_year'] = show_title['original_release_year']
    else:
        show_dict['release_year'] = None
        
    show_dict['type'] = show_title['object_type']
    show_dict['imdb_popularity'] = None
    show_dict['tmdb_popularity'] = None
    show_dict['imdb_score'] = None
    show_dict['imdb_votes'] = None
    show_dict['tmdb_score'] = None
    
    try:
        # runs a loop on scoring to get the proper values
        for item in show_title['scoring']:
            if item['provider_type'] == 'imdb:popularity':
                show_dict['imdb_popularity'] = item['value']

            if item['provider_type'] == 'tmdb:popularity':
                show_dict['tmdb_popularity'] = item['value']

            if item['provider_type'] == 'imdb:score':
                show_dict['imdb_score'] = item['value']       

            if item['provider_type'] == 'imdb:votes':
                show_dict['imdb_votes'] = item['value']

            if item['provider_type'] == 'tmdb:score':    
                show_dict['tmdb_score'] = item['value']
    except:
        pass
    
    show_dict['imdb_id'] = None
    show_dict['tmdb_id'] = None

    # runs a for loop on external_ids to get the proper values
    for item in show_title['external_ids']:
        if item['provider'] == 'imdb_latest':
            show_dict['imdb_id'] = item['external_id']

        if item['provider'] == 'tmdb_latest':
            show_dict['tmdb_id'] = item['external_id']
            
    if 'genre_ids' in show_title.keys():
        show_dict['genre_ids'] = show_title['genre_ids']
    else:
        show_dict['genre_ids'] = None
    
    # checks if age_certification is in dictionary keys
    if 'age_certification' in show_title.keys():
        show_dict['age_certification'] = show_title['age_certification']
    else:
        show_dict['age_certification'] = None
        
    if 'runtime' in show_title.keys():
        show_dict['runtime'] = show_title['runtime']
    else:
        show_dict['runtime'] = None
    
    if 'production_countries' in show_title.keys():
        show_dict['production_countries'] = show_title['production_countries']
    else:
        show_dict['production_countries'] = None
    
    if 'seasons' in show_title.keys():
        show_dict['seasons'] = len(show_title['seasons'])
    else:
        show_dict['seasons'] = 0
    
    return show_dict

# Get Funimation Shows and Movies

### Shows

In [262]:
# for loop to run through each item from provider
# place it into a list of dictionaries
funimation_shows = []
for n in range(1,50):
    # runs the loop every 1 seconds
    time.sleep(1)
    results = just_watch.search_for_item(providers=['fmn'],
                                     page=n,
                                     content_types=['show'])
    
    # check if 'items' is empty
    if not results['items']:
        break
    else:
        # for loop to run through results['items']
        for show in results['items']:

            # sets the show id so it can used in the get_title method
            show_id = show['id']
            show_title = just_watch.get_title(title_id=show_id, content_type='show')

            show_dict = create_show_dict(show_title, show_id)

            # adds dictionary to show list
            funimation_shows.append(show_dict)

In [264]:
fun_shows_df = pd.DataFrame.from_dict(funimation_shows)

In [332]:
fun_shows_df.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989,show,805.0,387.966,8.8,128409.0,8.286,tt0121220,26447,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24,[JP],16
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013,show,55.0,89.689,9.0,325381.0,8.643,tt2560140,205148,"[1, 14, 2, 6, 7, 9]",TV-MA,24,[JP],4
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986,show,1936.0,15.964,8.6,56606.0,8.218,tt0088509,210469,"[2, 1, 3, 7, 14, 12]",TV-14,24,[JP],10


### Movies

In [268]:
# for loop to run through each item from provider
# place it into a list of dictionaries
funimation_movies = []
for n in range(1,50):
    # runs the loop every 1 seconds
    time.sleep(1)
    results = just_watch.search_for_item(providers=['fmn'],
                                     page=n,
                                     content_types=['movie'])
    
    # check if 'items' is empty
    if not results['items']:
        break
    else:
        # for loop to run through results['items']
        for show in results['items']:

            # sets the show id so it can used in the get_title method
            show_id = show['id']
            show_title = just_watch.get_title(title_id=show_id, content_type='movie')

            show_dict = create_show_dict(show_title, show_id)

            # adds dictionary to show list
            funimation_movies.append(show_dict)

In [269]:
fun_movies_df = pd.DataFrame.from_dict(funimation_movies)

In [333]:
fun_movies_df.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons
0,tm44887,44887,Akira,/poster/239922303/{profile},A secret military project endangers Neo-Tokyo ...,1988,movie,1964.0,43.587,8.0,185179.0,8.000,tt0094625,592646,"[1, 2, 6, 14, 7, 12]",R,124,[JP],0
1,tm842683,842683,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,/poster/176260460/{profile},"Tanjirō Kamado, joined with Inosuke Hashibira,...",2020,movie,115.0,772.436,8.2,55979.0,8.323,tt11032374,635302,"[2, 1, 7, 12, 9]",R,117,[JP],0
2,tm139269,139269,Little Nemo: Adventures in Slumberland,/poster/263027719/{profile},A little boy whose dreams transcend reality is...,1989,movie,NaN,9.713,7.1,7493.0,6.700,tt0104740,22611,"[10, 7, 1, 2, 6, 3]",G,95,"[JP, US]",0


In [279]:
funimation_titles = fun_shows_df.append(fun_movies_df)

In [282]:
funimation_titles.shape

(848, 19)

In [283]:
# adding streaming_app column to dataframe
funimation_titles['streaming_app'] = 'Funimation'

In [284]:
funimation_titles.shape

(848, 20)

In [335]:
funimation_titles.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989,show,805.0,387.966,8.8,128409.0,8.286,tt0121220,26447,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24,[JP],16,Funimation
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013,show,55.0,89.689,9.0,325381.0,8.643,tt2560140,205148,"[1, 14, 2, 6, 7, 9]",TV-MA,24,[JP],4,Funimation
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986,show,1936.0,15.964,8.6,56606.0,8.218,tt0088509,210469,"[2, 1, 3, 7, 14, 12]",TV-14,24,[JP],10,Funimation


## Crunchyroll Shows and Movies

### Shows

In [307]:
# for loop to run through each item from provider
# place it into a list of dictionaries
counter = 1
crunchyroll_shows = []
for n in range(1,50):
    print(counter)
    counter += 1
    # runs the loop every 1 seconds
    time.sleep(2)
    results = just_watch.search_for_item(providers=['cru'],
                                     page=n,
                                     content_types=['show'])
    
    # check if 'items' is empty
    if not results['items']:
        break
    else:
        # for loop to run through results['items']
        for show in results['items']:

            # sets the show id so it can used in the get_title method
            show_id = show['id']
            show_title = just_watch.get_title(title_id=show_id, content_type='show')

            show_dict = create_show_dict(show_title, show_id)

            # adds dictionary to show list
            crunchyroll_shows.append(show_dict)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [308]:
crunchyroll_shows_df = pd.DataFrame.from_dict(crunchyroll_shows)

In [309]:
crunchyroll_shows_df.shape

(1092, 19)

In [336]:
crunchyroll_shows_df.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989.0,show,805.0,387.966,8.8,128409.0,8.286,tt0121220,26447,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24.0,[JP],16
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013.0,show,55.0,89.689,9.0,325381.0,8.643,tt2560140,205148,"[1, 14, 2, 6, 7, 9]",TV-MA,24.0,[JP],4
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986.0,show,1936.0,15.964,8.6,56606.0,8.218,tt0088509,210469,"[2, 1, 3, 7, 14, 12]",TV-14,24.0,[JP],10


### Movies

In [314]:
# for loop to run through each item from provider
# place it into a list of dictionaries
crunchyroll_movies = []
for n in range(1,50):
    # runs the loop every 1 seconds
    time.sleep(2)
    results = just_watch.search_for_item(providers=['cru'],
                                     page=n,
                                     content_types=['movie'])
    
    # check if 'items' is empty
    if not results['items']:
        break
    else:
        # for loop to run through results['items']
        for show in results['items']:

            # sets the show id so it can used in the get_title method
            show_id = show['id']
            show_title = just_watch.get_title(title_id=show_id, content_type='movie')

            show_dict = create_show_dict(show_title, show_id)

            # adds dictionary to show list
            crunchyroll_movies.append(show_dict)

In [315]:
crunchyroll_movies_df = pd.DataFrame(crunchyroll_movies)

In [316]:
crunchyroll_movies_df.shape

(1, 19)

In [317]:
crunchyroll_titles = crunchyroll_shows_df.append(crunchyroll_movies_df)

In [318]:
crunchyroll_titles.shape

(1093, 19)

In [320]:
# adding streaming_app column
crunchyroll_titles['streaming_app'] = 'Crunchyroll'

In [337]:
crunchyroll_titles.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989.0,show,805,387.966,8.8,128409.0,8.286,tt0121220,26447,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24.0,[JP],16,Crunchyroll
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013.0,show,55,89.689,9.0,325381.0,8.643,tt2560140,205148,"[1, 14, 2, 6, 7, 9]",TV-MA,24.0,[JP],4,Crunchyroll
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986.0,show,1936,15.964,8.6,56606.0,8.218,tt0088509,210469,"[2, 1, 3, 7, 14, 12]",TV-14,24.0,[JP],10,Crunchyroll


## Combing Funimation and Crunchyroll

In [321]:
titles = funimation_titles.append(crunchyroll_titles)

In [322]:
titles.shape

(1941, 20)

In [338]:
titles.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989.0,show,805,387.966,8.8,128409.0,8.286,tt0121220,26447,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24.0,[JP],16,Funimation
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013.0,show,55,89.689,9.0,325381.0,8.643,tt2560140,205148,"[1, 14, 2, 6, 7, 9]",TV-MA,24.0,[JP],4,Funimation
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986.0,show,1936,15.964,8.6,56606.0,8.218,tt0088509,210469,"[2, 1, 3, 7, 14, 12]",TV-14,24.0,[JP],10,Funimation


In [330]:
# save titles to csv
titles.to_csv('./data/titles.csv')